# Pipeline de Series Temporales: Carga, Transformación y Feature Engineering

Este notebook encapsula el proceso de carga de datos, transformación a series temporales y creación de características y target en funciones reutilizables para forecasting.

In [11]:
from src.data_utils import load_raw_data # type: ignore

# Descarga y carga datos raw desde BigQuery para el rango de fechas especificado
df_raw = load_raw_data(
    fecha_inicio="2023-01-02", 
    fecha_fin="2025-06-30", 
    descargar_bq=False)

df_raw.describe()

Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250630.parquet
Total de fechas faltantes: 4
Fechas faltantes: ['2023-12-25T00:00:00.000000000' '2024-01-01T00:00:00.000000000'
 '2024-12-25T00:00:00.000000000' '2025-01-01T00:00:00.000000000']


,fecha,cantidad,base_imponible,tipo_IVA,total,is_summer_peak,is_easter
count,337353,337353.000000,337353.000000,335106.000000,337353.000000,337353.000000,337353.0
mean,2024-03-10 00:54:30.032281344,1.209303,2.010886,9.084558,2.199180,0.208473,0.0
min,2023-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,2023-07-28 00:00:00,1.000000,1.350000,10.000000,1.450000,0.000000,0.0
50%,2024-03-09 00:00:00,1.000000,1.640000,10.000000,1.800000,0.000000,0.0
75%,2024-09-25 00:00:00,1.000000,2.320000,10.000000,2.550000,0.000000,0.0
max,2025-06-30 00:00:00,93.000000,287.270000,10.000000,316.000000,1.000000,0.0
std,NaN,0.733263,1.692278,2.793367,1.862156,0.406217,0.0


In [14]:
from src.data_utils import guardar_time_series_interim # type: ignore

def transformar_a_series_temporales(
    df_raw,
    fecha_inicio='2023-01-02',
    fecha_fin='2025-06-29',
    familia='BOLLERIA',
    output_path=None,
    min_dias_semana=7,
    guardar_interim=True,  # Nuevo parámetro
    interim_dir='data/interim'
):
    import pandas as pd

    """
    Limpia, homogeneiza y agrega los datos diarios a series semanales completas para la familia indicada.

    Parámetros:
    - df_raw: DataFrame con datos crudos
    - fecha_inicio: Fecha de inicio (str o datetime)
    - fecha_fin: Fecha fin (str o datetime)
    - familia: Familia de productos a filtrar (str)
    - output_path: Path opcional para guardar el resultado (str o Path)
    - min_dias_semana: Mínimo de días para considerar una semana (int, por defecto 7)
    - guardar_interim: Si True, guarda el DataFrame en data/interim
    - interim_dir: Carpeta donde guardar el archivo interim

    Retorna:
    - DataFrame con series temporales semanales
    """
    df = df_raw.copy()
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Filtrar rango de fechas
    df = df[(df['fecha'] >= fecha_inicio) & (df['fecha'] <= fecha_fin)]
    # Homogeneizar familia si es necesario (ejemplo: 'BEBIDA' a 'BEBIDAS')
    if 'familia' in df.columns:
        df.loc[df['familia'] == 'BEBIDA', 'familia'] = 'BEBIDAS'
    # Imputar valores nulos básicos
    for col in ['base_imponible', 'total']:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    # Asegurar columnas exógenas
    if 'is_summer_peak' not in df.columns:
        df['is_summer_peak'] = 0
    if 'is_easter' not in df.columns:
        df['is_easter'] = 0
    # Calcular semana ISO
    iso = df['fecha'].dt.isocalendar()
    df['year_iso'] = iso['year']
    df['week_iso'] = iso['week']

    # Contar días únicos por semana/familia
    conteo_dias = df.groupby(['year_iso','week_iso','familia'])['fecha'].nunique().reset_index(name='dias_semana')
    # Agregación semanal
    df_semanal = (
        df.groupby(['year_iso','week_iso','familia'], as_index=False)
          .agg({
             'base_imponible': 'sum',
             'is_summer_peak': 'max',
             'is_easter':      'max'
          })
        .merge(conteo_dias, on=['year_iso','week_iso','familia'])
    )
    # Filtrar solo semanas con el mínimo de días
    df_semanal = df_semanal[df_semanal['dias_semana'] >= min_dias_semana]
    # Filtrar familia
    df_familia_semanal = df_semanal.query(f"familia=='{familia}'").copy()
    df_familia_semanal.rename(columns={'year_iso':'year','week_iso':'week'}, inplace=True)
    df_familia_semanal = df_familia_semanal.sort_values(['year','week']).reset_index(drop=True)

    # Guardar el resultado si se proporciona un path de salida
    if output_path:
        df_familia_semanal.to_parquet(str(output_path), index=False)
        print(f"Series temporales guardadas en: {output_path}")
    # Guardar en interim si se solicita
    if guardar_interim:
        guardar_time_series_interim(df_familia_semanal, familia, interim_dir)

    return df_familia_semanal

# Ejemplo de uso:
df_familia_semanal = transformar_a_series_temporales(df_raw, familia='BOLLERIA', guardar_interim=True)

# Supón que ya tienes df_raw cargado
df_familia_semanal = transformar_a_series_temporales(df_raw)

# Visualiza las primeras filas
df_familia_semanal.head(150)

Archivo guardado en: data/interim\time_series_BOLLERIA_weekly_20250730_174037.parquet
Archivo guardado en: data/interim\time_series_BOLLERIA_weekly_20250730_174037.parquet


,year,week,familia,base_imponible,is_summer_peak,is_easter,dias_semana
0,2023,1,BOLLERIA,825.11,0,0,7
1,2023,2,BOLLERIA,658.40,0,0,7
2,2023,3,BOLLERIA,741.40,0,0,7
3,2023,4,BOLLERIA,653.64,0,0,7
4,2023,5,BOLLERIA,680.46,0,0,7
...,...,...,...,...,...,...,...
121,2025,22,BOLLERIA,802.16,0,0,7
122,2025,23,BOLLERIA,881.72,0,0,7
123,2025,24,BOLLERIA,1015.97,0,0,7
124,2025,25,BOLLERIA,1014.76,0,0,7


In [15]:
df_familia_semanal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            126 non-null    UInt32 
 1   week            126 non-null    UInt32 
 2   familia         126 non-null    object 
 3   base_imponible  126 non-null    float64
 4   is_summer_peak  126 non-null    int64  
 5   is_easter       126 non-null    int64  
 6   dias_semana     126 non-null    int64  
dtypes: UInt32(2), float64(1), int64(3), object(1)
memory usage: 6.3+ KB
